In [1]:
# import package
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, ConcatDataset
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.utils as vutils
from torchvision.datasets import DatasetFolder 
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from RealESRGAN import RealESRGAN
import os
import torchattacks
from models_c import *
from tqdm import tqdm
import cv2

# for denoise
from utils import utils_logger
from utils import utils_model
from utils import utils_image as util
from collections import OrderedDict

# check CUDA status
!nvidia-smi
print("CUDA Available : ", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

noise_color = 'w'

Thu May 25 11:28:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:2B:00.0  On |                  N/A |
| 46%   52C    P8    42W / 350W |    332MiB / 24576MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Preprocessing

train_tfm = transforms.Compose([
    transforms.RandomCrop((256, 256)),
    transforms.ToTensor(),
])

toPillow = transforms.ToPILImage()
totensor = transforms.ToTensor()

In [3]:
model = models.densenet201(weights='DenseNet201_Weights.DEFAULT')
model = nn.Sequential(*list(model.children())[:-1],ClassifierH2())
model = model.to(device)
checkpoint = torch.load(f'./pretrain/flower102_2023052039.pt')
model.load_state_dict(checkpoint)
model.eval()  
print('load model : flower102')

load model : flower102


In [4]:
test_set = DatasetFolder('flower102/test', loader=lambda x: Image.open(x), extensions="jpg", transform = train_tfm)

test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=0)

Debugger :  class -> idx 
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17, '18': 18, '19': 19, '20': 20, '21': 21, '22': 22, '23': 23, '24': 24, '25': 25, '26': 26, '27': 27, '28': 28, '29': 29, '30': 30, '31': 31, '32': 32, '33': 33, '34': 34, '35': 35, '36': 36, '37': 37, '38': 38, '39': 39, '40': 40, '41': 41, '42': 42, '43': 43, '44': 44, '45': 45, '46': 46, '47': 47, '48': 48, '49': 49, '50': 50, '51': 51, '52': 52, '53': 53, '54': 54, '55': 55, '56': 56, '57': 57, '58': 58, '59': 59, '60': 60, '61': 61, '62': 62, '63': 63, '64': 64, '65': 65, '66': 66, '67': 67, '68': 68, '69': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '80': 80, '81': 81, '82': 82, '83': 83, '84': 84, '85': 85, '86': 86, '87': 87, '88': 88, '89': 89, '90': 90, '91': 91, '92': 92, '93': 93, '94': 94, '95': 95, '96': 96, '97': 97, '98': 98, '99

In [5]:
def denoise(imgs, model_name = 'drunet_color',x8 = False, show_img = False, border = 0, n_channels = 3, model_pool = 'denoise_model'):
    # set params
    noise_level_model = 15
    model_name = model_name
    testset_name = 'bsd68'

    #set result path
    results = 'results'                  # fixed
    task_current = 'dn'                  # 'dn' for denoising
    result_name = testset_name + '_' + task_current + '_' + model_name
    model_path = os.path.join(model_pool, model_name+'.pth')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.empty_cache()

    # initial model
    from models_c.network_unet import UNetRes as net
    model = net(in_nc=n_channels+1, out_nc=n_channels, nc=[64, 128, 256, 512], nb=4, act_mode='R', downsample_mode="strideconv", upsample_mode="convtranspose")
    model.load_state_dict(torch.load(model_path), strict=True)
    model.eval()
    for k, v in model.named_parameters():
        v.requires_grad = False
    model = model.to(device)
    number_parameters = sum(map(lambda x: x.numel(), model.parameters()))

    # params show
    test_results = OrderedDict()
    test_results['psnr'] = []
    test_results['ssim'] = []
        
    img = toPillow(imgs)
    img.save('mask/2.bmp')
    img_H = util.imread_uint("mask/2.bmp", n_channels=n_channels)
    img_L = util.uint2single(img_H)
    #print(img_H.shape)
    # Add noise without clipping
    np.random.seed(seed=0)  # for reproducibility
    img_L += np.random.normal(0, noise_level_model/255., img_L.shape)

    util.imshow(util.single2uint(img_L), title='Noisy image with noise level {}'.format(noise_level_model)) if show_img else None

    img_L = util.single2tensor4(img_L)
    img_L = torch.cat((img_L, torch.FloatTensor([noise_level_model/255.]).repeat(1, 1, img_L.shape[2], img_L.shape[3])), dim=1)
    #print(img_L.shape)
    img_L = img_L.to(device)
        
        #print(img_L.size(2), img_L.size(3))

    if not x8 and img_L.size(2)//8==0 and img_L.size(3)//8==0:
        img_E = model(img_L)
    elif not x8 and (img_L.size(2)//8!=0 or img_L.size(3)//8!=0):
        img_E = utils_model.test_mode(model, img_L, refield=64, mode=5)
    elif x8:
        img_E = utils_model.test_mode(model, img_L, mode=3)

    img_E = util.tensor2uint(img_E)
    #print(img_E.shape)

    if n_channels == 1:
        img_H = img_H.squeeze() 
    psnr = util.calculate_psnr(img_E, img_H, border=border)
    ssim = util.calculate_ssim(img_E, img_H, border=border)
    test_results['psnr'].append(psnr)
    test_results['ssim'].append(ssim)

    util.imsave(img_E, "mask/3.bmp")
        
    img_after = Image.open("mask/3.bmp")
    img_after = totensor(img_after)
    imgs = img_after

    ave_psnr = sum(test_results['psnr']) / len(test_results['psnr'])
    ave_ssim = sum(test_results['ssim']) / len(test_results['ssim'])
    #print('Average PSNR/SSIM(RGB) - {} - PSNR: {:.2f} dB; SSIM: {:.4f}'.format(result_name, ave_psnr, ave_ssim))

    return imgs

In [6]:
import cv2

def randomNoise(e, imgs):
    for i in range(imgs.shape[0]):
        img = imgs[i]
        mask = torch.zeros((img.shape[0], img.shape[1], img.shape[2])).to(device)
        points = []
        #print(mask.shape, img.shape)
        for k in range(e):
            m_i = int(np.random.uniform(0, img.shape[1])) 
            m_j = int(np.random.uniform(0, img.shape[2]))
            mask[:, m_j, m_i] = img[:, m_j, m_i]
            points.append([m_j, m_i])
            img[0, m_j, m_i] = (1 if noise_color == 'w' else 25/255)
            img[1, m_j, m_i] = (1 if noise_color == 'w' else 20/255)
            img[2, m_j, m_i] = (1 if noise_color == 'w' else 20/255)
        new_img = toPillow(mask)
        new_img.save('mask/0.bmp')
        img = denoise(img).to(device)
        for point in points:
            img[:, point[0], point[1]] = mask[:, point[0], point[1]]
        imgs[i] = img
    return imgs

In [7]:
# import model

path = f'flower102_test_set_{noise_color}'

if not os.path.isdir(path):
    os.mkdir(path)

idx = 0

for batch in tqdm(test_loader):
    imgs, labels = batch
    imgs, labels = imgs.to(device), labels.to(device)
    
    imgs = randomNoise(98, imgs)
    #imgs = denoise(imgs)
    for i in range(imgs.shape[0]):
        img = imgs[i]
        new_img = toPillow(img).convert('RGB')
        if not os.path.isdir(path + '/' + str(int(labels[i]))):
            os.mkdir(path + '/' + str(int(labels[i])) + '/')
        new_img.save(path + '/' + str(int(labels[i])) + '/preprocess_' + str(idx) + '.jpg')
        idx += 1
torch.cuda.empty_cache()

100%|██████████| 26/26 [04:02<00:00,  9.34s/it]


In [8]:
# import model
from tqdm import tqdm

path = f'flower_test_adv_set_{noise_color}'
criterion = nn.CrossEntropyLoss()
pgd_attack = torchattacks.PGD(model, eps=8/255, steps = 40)

#adversary = LinfPGDAttack(model)

if not os.path.isdir(path):
    os.mkdir(path)

idx = 0

#test = True
for batch in tqdm(test_loader):
    imgs, labels = batch
    imgs, labels = imgs.to(device), labels.to(device)
    imgs = pgd_attack(imgs, labels)
    torch.cuda.empty_cache()
    
    imgs = randomNoise(98, imgs)
    #imgs = denoise(imgs)
    for i in range(imgs.shape[0]):
        img = imgs[i]
        new_img = toPillow(img)
        if not os.path.isdir(path + '/' + str(int(labels[i]))):
            os.mkdir(path + '/' + str(int(labels[i])) + '/')
        new_img.save(path + '/' + str(int(labels[i])) + '/adv_' + str(idx) + '.jpg')
        idx += 1

100%|██████████| 26/26 [06:03<00:00, 13.96s/it]
